# Magnetic Dipole Approximation Demo

This demo verifies the magnetic dipole approximation. It compares the magnetic field from a small, circular current loop at a large distance with the analytical formula for a magnetic dipole.

The key verification is to show that the error between the numerically computed field from the loop and the analytical dipole field decreases as the distance from the loop increases. We expect the error to fall off, and we will visualize this relationship with a plot.

In [ ]:
import numpy as np
from MTFLibrary import *
from MTFLibrary.EMLibrary.biot_savart import serial_biot_savart
from MTFLibrary.EMLibrary.current_ring import current_ring
from MTFLibrary.taylor_function import MultivariateTaylorFunctionBase
import math
import matplotlib.pyplot as plt

# --- Global MTF Settings ---
# max_dimension must be at least 4 because current_ring uses Var(4) internally
# to parameterize the segments for integration.
initialize_mtf_globals(max_order=3, max_dimension=4)
set_global_etol(1e-12)

# --- Define Variables for MTF ---
x = Var(1)
y = Var(2)
z = Var(3)

### Function to Calculate Approximation Error

We define a function to compute the error between the numerical and analytical fields for a given distance.

In [ ]:
def calculate_error_at_distance(distance):
    # --- 1. Define a small current loop ---
    ring_radius = 0.01
    num_segments_ring = 20
    current = 1.0
    ring_center_point = np.array([0.0, 0.0, 0.0])
    ring_axis_direction = np.array([0, 0, 1])

    segment_mtfs, element_lengths, direction_vectors = current_ring(
        ring_radius, num_segments_ring, ring_center_point, ring_axis_direction)

    # --- 2. Define a far-field point ---
    field_point_mtf = np.array([[x, y + distance, z]], dtype=object)

    # --- 3. Calculate the B-field from the loop (Numerical) ---
    B_numerical_mtf_with_u = serial_biot_savart(segment_mtfs, element_lengths, direction_vectors, field_point_mtf)
    B_numerical_mtf = [integrate(bfld, 4, -1, 1) for bfld in B_numerical_mtf_with_u[0]]

    # --- 4. Calculate the B-field from the analytical dipole formula ---
    mu_0_4pi = 1e-7
    area = math.pi * ring_radius**2
    m_vec = np.array([0, 0, current * area])
    m_mtf = np.array([MultivariateTaylorFunctionBase.from_constant(c) for c in m_vec], dtype=object)
    r_mtf = np.array([x, y + distance, z], dtype=object)
    r_squared_mtf = r_mtf[0]**2 + r_mtf[1]**2 + r_mtf[2]**2
    r_mag_mtf = sqrt_taylor(r_squared_mtf)
    m_dot_r_mtf = m_mtf[0]*r_mtf[0] + m_mtf[1]*r_mtf[1] + m_mtf[2]*r_mtf[2]
    term1_scalar_inv_r5 = 3 * m_dot_r_mtf / (r_mag_mtf**5)
    term1 = [c * term1_scalar_inv_r5 for c in r_mtf]
    term2_scalar_inv_r3 = 1 / (r_mag_mtf**3)
    term2 = [c * term2_scalar_inv_r3 for c in m_mtf]
    B_analytic_mtf = [mu_0_4pi * (t1 - t2) for t1, t2 in zip(term1, term2)]

    # --- 5. Calculate the error ---
    # We use the norm of the difference of the constant terms (the field at the expansion point)
    B_num_const = np.array([c.extract_coefficient((0,0,0,0)) for c in B_numerical_mtf])
    B_an_const = np.array([c.extract_coefficient((0,0,0,0)) for c in B_analytic_mtf])
    error = np.linalg.norm(B_num_const - B_an_const)
    return error

### Run Simulation Over a Range of Distances

In [ ]:
distances = np.logspace(0, 2, 20) # from 1m to 100m
errors = []
for d in distances:
    error = calculate_error_at_distance(d)
    print(f"Distance: {d:.2f}, Error: {error:.2e}")
    errors.append(error)

### Plot Error vs. Distance

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(distances, errors, 'o-')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Distance from loop (m)')
plt.ylabel('Approximation Error (Norm of B-field difference)')
plt.title('Dipole Approximation Error vs. Distance')
plt.grid(True, which="both", ls="--")
plt.show()